# Data Preprocessing - MLP

Hand-crafted filtering, data distribution 을 이용한 filtering 방식과 다르게 공정 시작 시간을 이용해서 작업과 작업 사이의 쉬는 시간을 파악해 작업 시간을 직접 조정한다.

Step 5 data는 공정 시작 시간, 공정 완료 시간이 존재하지만 나머지 step들에 대해서는 공정 시작 시간 밖에 존재하지 않는다. 그래서 우리는 다음 파이프의 공정 시작 시간에서 현재 파이프의 공정 시작 시간의 차이를 작업 시간으로 간주했지만 이는 실질적으로 공정 시작, 완료 시간 사이의 쉬는 시간에 대해서는 고려하지 못하기 때문에 정확한 공정 시간이라고 할 수 없고 실제로 regression 수행 시 큰 error 값을 가진다.

이를 위해 기존의 data에서 outlier를 filtering 하는 방식보다는 작업 시간을 조정해주는 방법을 고려해 보았다.


먼저 Step5 data에 대해서 공정 시작 시간을 기준으로 오름차순을 해준다.

MLP의 input은 $i$ 번째 파이프와 $i+1$ 번째 파이프의 공정 시작 시간이다.
Output은 $i+1$ 번째 파이프의 공정 시작 시간과 $i$ 번째 파이프의 공정 완료 시간 즉, 현재 파이프와 다음 파이프 공정 사이의 쉬는 시간을 의미한다.

만약 모델이 잘 학습 된다면 $i$ 번째 파이프와 $i+1$ 번째 파이프의 공정 시작 시간으로 이 두 파이프 공정 사이의 쉬는 시간을 알 수 있을 것이고 우리가 기존에 사용했던

 $$작업\ 시간_i = i+1\ 번째\ 파이프의\ 공정\ 시작\ 시간 - i\ 번째\ 파이프의\ 공정\ 시작\ 시간$$

에서 MLP output 값을 빼주면 $i$ 번째 파이프의 공정 시간에서 쉬는 시간까지 고려해 조정할 수 있을 것이다.


$$작업\ 시간_i = i+1\ 번째\ 파이프의\ 공정\ 시작\ 시간 - i\ 번째\ 파이프의\ 공정\ 시작\ 시간 - \text{MLP}의\ \text{output}\ (쉬는\ 시간)$$

In [ ]:
# Work directory 이동
%cd /content/drive/MyDrive/pipe

/content/drive/MyDrive/pipe


In [ ]:
# Library import
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
data = pd.read_excel('regression.xlsx')

# [input1, input2], output
X = data[['new_input1', 'new_input2']].values  # new_input은 기존의 timestampe type의 input을 int type으로 바꿔준 형태이다.
y = data['output'].values  # output : 쉬는 시간 (= 다음 파이프의 시작 시간 - 현재 파이프의 완료 시간)


# Data scaling (StandardScaler)
scaler = StandardScaler()
X = scaler.fit_transform(X)


# Data -> Tensor
X_train_tensor = torch.tensor(X, dtype=torch.float32)
y_train_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)


# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# MLP Model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(2, 64)  # input layer (2 features)
        self.layer2 = nn.Linear(64, 64)  # hidden layer
        self.output_layer = nn.Linear(64, 1)  # output layer (1 output)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

In [ ]:
# Model, Optimizer, Loss
model = MLP()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MLP(
  (layer1): Linear(in_features=2, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
# 9. Model Train
def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.to(device))
            loss = criterion(outputs, targets.to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

In [ ]:
# 11. Train!!
train(model, train_loader, criterion, optimizer, epochs=1000)

Epoch [1/1000], Loss: 1234616.1427
Epoch [2/1000], Loss: 1234618.1716
Epoch [3/1000], Loss: 1234758.4787
Epoch [4/1000], Loss: 1234115.8781
Epoch [5/1000], Loss: 1233801.8856
Epoch [6/1000], Loss: 1233501.1005
Epoch [7/1000], Loss: 1233188.5448
Epoch [8/1000], Loss: 1232935.0287
Epoch [9/1000], Loss: 1232779.7470
Epoch [10/1000], Loss: 1233110.2922
Epoch [11/1000], Loss: 1232663.7523
Epoch [12/1000], Loss: 1232419.9566
Epoch [13/1000], Loss: 1232352.4330
Epoch [14/1000], Loss: 1232217.2509
Epoch [15/1000], Loss: 1232219.6423
Epoch [16/1000], Loss: 1232015.6219
Epoch [17/1000], Loss: 1231893.2074
Epoch [18/1000], Loss: 1231927.4129
Epoch [19/1000], Loss: 1231597.1921
Epoch [20/1000], Loss: 1231422.0630
Epoch [21/1000], Loss: 1231233.8469
Epoch [22/1000], Loss: 1231058.0428
Epoch [23/1000], Loss: 1230803.0114
Epoch [24/1000], Loss: 1230571.3209
Epoch [25/1000], Loss: 1230378.4304
Epoch [26/1000], Loss: 1230050.5474
Epoch [27/1000], Loss: 1229749.8748
Epoch [28/1000], Loss: 1229422.0366
E

In [ ]:
torch.save(model.state_dict(), 'pipe_model.pth') # Save model

In [ ]:
# Stpe 5를 제외한 모든 step에 적용
test_df = pd.read_excel('regression_step6.xlsx') # 예시 : step 6

X_test = test_df[['new_input1', 'new_input2']].values
y_test = test_df['작업 시간'].values

scaler_test = StandardScaler()
X_test = scaler_test.fit_transform(X_test)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# 모델 테스트 함수 (output 값 저장)
def test(model, test_loader):
    model.load_state_dict(torch.load('pipe_model.pth'))  # 모델 가중치 불러오기
    model.eval()
    outputs_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs.to(device))

            # outputs 값을 리스트에 저장
            outputs_list.extend(outputs.cpu().numpy())  # GPU에서 CPU로 변환 후 numpy로 변환하여 저장

    return [round(output.item(), 1) for output in outputs_list]

In [ ]:
outputs = test(model, test_loader)

<ipython-input-55-549b2b57cac8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('pipe_model.pth'))  # 모델 가중치 불러오기


In [ ]:
test_df['output'] = outputs

In [ ]:
outputs

[-21.7,
 103.6,
 133.9,
 56.8,
 57.5,
 56.4,
 53.7,
 0.1,
 110.5,
 45.4,
 -2.8,
 32.7,
 1.1,
 95.1,
 16.6,
 4.4,
 3.4,
 6.6,
 12.8,
 4.2,
 13.6,
 2.7,
 234.6,
 9.5,
 5.0,
 8.5,
 22.4,
 2.5,
 83.0,
 44.3,
 7.3,
 6.5,
 13.0,
 6.1,
 -5.0,
 15.8,
 6.4,
 12.3,
 2.8,
 161.8,
 5.1,
 -1.8,
 -3.2,
 -9.5,
 -5.8,
 -10.3,
 62.6,
 -3.3,
 4.2,
 1.9,
 2.0,
 16.8,
 -3.7,
 -2.3,
 8.1,
 6.9,
 9.3,
 24.2,
 33.2,
 19.1,
 6.4,
 4484.3,
 -4.4,
 -1.5,
 -1.8,
 6.0,
 7.8,
 3.9,
 11.6,
 -8.8,
 60.0,
 12.7,
 2.2,
 -0.8,
 -6.8,
 -3.6,
 2.3,
 5.4,
 -12.0,
 0.1,
 -5.0,
 -6.1,
 54.1,
 -12.8,
 -21.2,
 -13.9,
 -12.0,
 189.1,
 -14.5,
 -6.3,
 -9.7,
 -12.5,
 -14.0,
 -14.8,
 -13.3,
 -12.2,
 -6.7,
 97.2,
 -13.0,
 -12.4,
 -14.7,
 -12.4,
 -11.9,
 -10.3,
 -22.2,
 5.4,
 -12.1,
 -16.0,
 -13.0,
 -13.9,
 -3.8,
 -13.3,
 -13.8,
 -13.4,
 26.9,
 -12.1,
 -13.0,
 -13.8,
 -11.6,
 16.0,
 -10.7,
 -12.5,
 -14.1,
 -12.4,
 -8.8,
 -2.0,
 25.6,
 -12.5,
 -13.2,
 -4.3,
 -12.3,
 -9.3,
 -5.9,
 -12.4,
 19.6,
 -10.8,
 -4.9,
 -12.6,
 -11.2,
 -9.0,
 5

Output을 보면 알겠지만 값이 음수로 나오는 것들이 있다. 이런 경우는 엑셀 파일에서 따로 0으로 처리해 주었다.

또한 작업 시간을 조정 했을 때, 음수가 나오는 경우가 아주 간혹 있는데 이 경우에는 값을 조정하기 전에 있는 값을 그대로 사용했다.

In [ ]:
# output 값 열을 추가한 뒤 엑셀 파일로 저장
test_df.to_excel('step6_preprocessing.xlsx')